## 本程序对已经挑选出cluster、expWT等文件的"_FLpick"文件夹数据，从中挑选出不符合定义要求的数据。本程序不统计label值。

### 假设需要统计的文件的目录结构为“AuPd-4000_au150-0_WT_untar_treat”（一级目录）——“500”（二级目录）——“cluster/coor/expWT”（三级目录），将本程序安放在与一级目录平行的另一个一级目录（或一级目录本身）的任意一个二级文件目录内，即可运行。
### 它会将不符合的数据，复制到与“cluster、coor、expWT”等平行目录的“check”文件夹中，并列出不符合数据的列表名称和独特“数字”编号

last update: 2022.5.27

contacts: zhaohf@ihep.ac.cn

# 配置环境

## 导入各种模块

In [1]:
import os
import sys
#import numpy as np
import pandas as pd
import shutil
import time
import datetime

输出导入模板的版本。

In [2]:
print('python version:',sys.version)
#print('numpy version:',np.__version__)
print('panda version:',pd.__version__)
#print('shutil version:',shutil.__version__)
#print('time version:',time.__version__)
#print('datetime version:',datetime.__version__)

python version: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
panda version: 1.4.2


# 设置输入文件夹和输出文件夹

## 设置源文件夹，即要处理的文件夹

In [3]:
# 设置源文件夹，即经过计算集群计算的文件夹
dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab'

## 设置可接受的label值范围

In [4]:
pick_dict = {
                 'cn1' : [3, 14]
                ,'cr1' : [2.5, 3.1]
}

## 设置放置被移除文件的文件夹

In [5]:
dir_check = 'check'
file_check = 'check.txt'

## 设置运行日志文件名

In [6]:
file_log = 'pick.log'

# 程序正文

## 设置工作文件夹

In [7]:
# 当前目录
path = os.path.abspath('./')
path

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\jupyterbook'

In [8]:
# 上一级目录
path_par = os.path.abspath(os.path.pardir)
path_par

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup'

In [9]:
# 设置源文件夹路径
path_src = os.path.join(path_par, dir_src)

## 检查源文件是否存在

In [10]:
# 检查源文件夹是否存在，存在则列出其内部文件夹，不存在则打印错误
exist = os.path.exists(path_src)
if exist:
    list_seed = os.listdir(path_src)
else:
    print()
    print('** Error!! WT untared directory', dir_src, 'is not found!' )
    print()
#    sys.exit()
list_seed

['100',
 '1000',
 '1100',
 '1200',
 '1300',
 '1400',
 '1500',
 '1600',
 '1700',
 '1800',
 '1900',
 '200',
 '2000',
 '2100',
 '2200',
 '2300',
 '2400',
 '2500',
 '2600',
 '2700',
 '2800',
 '2900',
 '300',
 '3000',
 '3100',
 '3200',
 '3300',
 '3400',
 '3500',
 '3600',
 '3700',
 '3800',
 '3900',
 '400',
 '4000',
 '500',
 '600',
 '700',
 '800',
 '900',
 'modify.log',
 'modify2.log',
 'pick.log']

In [11]:
# 去除list_seed中不是文件夹的部分
file_del = []
for dir_seed in list_seed:
    path_seed_src = os.path.join(path_src, dir_seed)
    if os.path.isfile(path_seed_src):
        file_del.append(dir_seed)
for file in file_del:
    list_seed.remove(file)

## 创建日志文件

In [12]:
#file_log = 'pick.log'
path_file_log = os.path.join(path_src, file_log)
flog = open(path_file_log, 'w')

## 数据筛选

In [13]:
#starttime = time.strftime("%Y_%m_%d-%H_%M_%S")
starttime1 = datetime.datetime.now()
#starttime2 = time.time()
##starttime3 = time.clock()
#print(f"== Start: {starttime}")
print(f"== Start: {starttime1}")
#print(f"== Start: {starttime2}")
##print(f"== Start: {starttime3}")
#print(f"== Start: {starttime}", file=flog)
print(f"== Start: {starttime1}", file=flog)
#print(f"== Start: {starttime2}", file=flog)
##print(f"== Start: {starttime3}", file=flog)
print()
print(file=flog)
#  N_bad 表示不好的位点数量（也就是配位数超过m的位点数）
N_bad = 0
# seed_not_check 包含没有check的seed
seed_not_check = []
# 进入各个cluster，即100、200等
for dir_seed in list_seed:
# 源seed的路径
    path_seed_src = os.path.join(path_src, dir_seed)
# 创建“check”文件夹目录
    path_dir_check = os.path.join(path_seed_src, dir_check)
#
#  检查是否已经存在check文件夹，若有，删除其中文件；若没有则构建
    os.makedirs(path_dir_check, exist_ok=True)
    shutil.rmtree(path_dir_check)
    os.makedirs(path_dir_check, exist_ok=True)
# 创建“check.dat”文件
    path_file_check = os.path.join(path_dir_check, file_check)
    fout = open(path_file_check, 'w')
# 先检查各seed下有没有pick_dict下所列的key文件夹，如果都没有，则报错，若有一部分，则继续运行
# pick_ok 表示到目前为止，是否有已经有pick_dict中的key找到，只要有一个就是True
    pick_ok = False
# number_list 用于存储不符合要求的（cn）数字
    number_list = []
    for dir_pick in pick_dict.keys():
#    for id, dir_seed in enumerate(list_seed):
        pick_data_min = pick_dict[dir_pick][0]
        pick_data_max = pick_dict[dir_pick][1]
#
        path_dir_pick = os.path.join(path_seed_src, dir_pick)
        exist = os.path.exists(path_dir_pick)
        if exist:
            pick_ok = True
#
#  在源文件夹内，查一下有哪些文件夹
            list_choose = os.listdir(path_seed_src)
# 源seed/dir_pick中各个pick文件  
            list_pick = os.listdir(path_dir_pick)
            for file_pick in list_pick:
# 各个cn1_i.dat文件的路径
                path_file_pick = os.path.join(path_dir_pick, file_pick)
#
#  读取cn1_i.dat中的信息
                pick_pd = pd.read_csv(path_file_pick,header=None)
                pick_data = pick_pd.values
#
                if pick_data < pick_data_min or pick_data > pick_data_max :
                    N_bad = N_bad + 1
                    print(f"** file {dir_seed} / {file_pick} copy to directory {dir_check}. **")
                    print(f"** file {dir_seed} / {file_pick} copy to directory {dir_check}. **", file=flog)
                    print(f"** Its data is {pick_data[0][0]} .")
                    print(f"** Its data is {pick_data[0][0]} .", file=flog)
#
#  将配位数大于m的文件——所有seed文件夹下各文件夹中的文件，如coor、cluster、expWT等——移动到check文件夹下
#                    try:
#                        shutil.move(path_file_pick,path_dir_check)
#                    except shutil.Error:
#                        path_del = os.path.join(path_dir_check, file_pick)
#                        os.remove(path_del)
#                        shutil.move(path_file_pick,path_dir_check)
#
#  对cn1_N.dat文件进行断句，找到是第几个位点，便于去cluster、expWT等其他文件夹中进行寻找。
                    result = os.path.splitext(file_pick)
                    start = len(dir_pick) + 1
                    number = result[0][start:]
                    if number not in number_list:
                        number_list.append(number)
                        print(f"{number:>4s} {file_pick:>}", end = '\n', file=fout)
                        shutil.copy2(path_file_pick,path_dir_check)
    #
                        key = '_' + number + '.'
                        for dest in list_choose:
                            if dest != dir_pick and dest != dir_check:
                                path_dest_src = os.path.join(path_seed_src, dest)
                                list_dest = os.listdir(path_dest_src)
                                found = False
                                for file in list_dest:
                                    if key in file:
                                        found = True
                                        break
                                if found:
                                    path_file_src = os.path.join(path_dest_src, file)
#                                try:
#                                    shutil.move(path_file_src,path_dir_check)
#                                except shutil.Error:
#                                    path_file_work = os.path.join(path_dir_check, file)
#                                    os.remove(path_file_work)
#                                    shutil.move(path_file_src,path_dir_check)
                                    shutil.copy2(path_file_src, path_dir_check)
                                    print(f"{number:>4s} {file:>}", end = '\n', file=fout)
#
        else:
            print()
            print(file=flog)
            print(f"** Error!! Directory {dir_pick} not found!")
            print(f"** Error!! Directory {dir_pick} not found!", file=flog)
            pick_ok = pick_ok or False
#        sys.exit()
#
    fout.close()
# 如果没找到需要check的点位，则不产生check文件
    if len(number_list) >0:
        result = os.path.splitext(file_check)
        file_checknum = result[0] + '_num' + result[1]
        path_file_checknum = os.path.join(path_dir_check, file_checknum)
        fout = open(path_file_checknum, 'w')
        for number in number_list:
            print(f"{number:>4s}", end = '\n', file=fout)
        fout.close()
    else:
        if os.path.exists(path_file_check):
            os.remove(path_file_check)
        seed_not_check.append(dir_seed)
#
    if pick_ok:
        print()
        print(file=flog)
        pass
    else:
        print()
        print(file=flog)
        print(f"** Error!! Directories {pick_dict.keys()} not found!")
        print(f"** Error!! Directories {pick_dict.keys()} not found!", file=flog)
        sys.exit()
#
# 删除 空文件夹
#
    if os.path.exists(path_dir_check):
        if len(os.listdir(path_dir_check)) == 0:
            os.rmdir(path_dir_check)
#
print()
print(file=flog)
print(f"** {N_bad} sites should be checked. **")
print(f"** {N_bad} sites should be checked. **", file=flog)
print()
print(file=flog)
print(f"** No sites in seeds {seed_not_check} will be checked. **")
print(f"** No sites in seeds {seed_not_check} will be checked. **", file=flog)
#
#endtime = time.strftime("%Y_%m_%d-%H_%M_%S")
endtime1 = datetime.datetime.now()
#endtime2 = time.time()
#endtime3 = time.clock()
#
print()
print(file=flog)
#print(f"== Start: {starttime}")
print(f"== Start: {starttime1}")
#print(f"== Start: {starttime2}")
##print(f"== Start: {starttime3}")
#print(f"== Start: {starttime}", file=flog)
print(f"== Start: {starttime1}", file=flog)
#print(f"== Start: {starttime2}", file=flog)
##print(f"== Start: {starttime3}", file=flog)
#print(f"== End: {endtime}")
print(f"== End: {endtime1}")
#print(f"== End: {endtime2}")
##print(f"== End: {endtime3}")
#print(f"== End: {endtime}", file=flog)
print(f"== End: {endtime1}", file=flog)
#print(f"== End: {endtime2}", file=flog)
##print(f"== End: {endtime3}", file=flog)
#
print(f"== Running time : {endtime1-starttime1}")
#print(f"== Running time : {endtime2-starttime2}")
##print(f"== Running time : {endtime3-starttime3}")
print(f"== Running time : {endtime1-starttime1}", file=flog)
#print(f"== Running time : {endtime2-starttime2}", file=flog)
##print(f"== Running time : {endtime3-starttime3}", file=flog)
flog.close()
#

== Start: 2022-05-27 23:29:15.430638

** file 100 / cn1_104.dat copy to directory check. **
** Its data is 16 .
** file 100 / cn1_25.dat copy to directory check. **
** Its data is 15 .
** file 100 / cn1_77.dat copy to directory check. **
** Its data is 18 .
** file 100 / cn1_80.dat copy to directory check. **
** Its data is 17 .
** file 100 / cr1_104.dat copy to directory check. **
** Its data is 3.175 .
** file 100 / cr1_143.dat copy to directory check. **
** Its data is 3.187 .
** file 100 / cr1_25.dat copy to directory check. **
** Its data is 3.106 .
** file 100 / cr1_77.dat copy to directory check. **
** Its data is 3.344 .
** file 100 / cr1_80.dat copy to directory check. **
** Its data is 3.23 .

** file 1000 / cn1_113.dat copy to directory check. **
** Its data is 2 .

** file 1100 / cn1_25.dat copy to directory check. **
** Its data is 144 .
** file 1100 / cn1_70.dat copy to directory check. **
** Its data is 42 .
** file 1100 / cr1_25.dat copy to directory check. **
** Its da